In [ ]:
import numpy as np
import emcee
import matplotlib.pyplot as plt

# Define the model
def model(params, x):
    slope, intercept, amplitude, period, phase = params
    return slope * x + intercept + amplitude * np.sin(2 * np.pi * x / period + phase)

# Define the log likelihood function
def log_likelihood(params, x, y, yerr):
    model_prediction = model(params, x)
    residuals = y - model_prediction
    chi_squared = np.sum(residuals**2 / yerr**2)
    return -0.5 * chi_squared

# Define the log prior function
def log_prior(params):
    slope, intercept, amplitude, period, phase = params
    # Flat priors
    if -100 < slope < 100 and -100 < intercept < 100 and 0 < amplitude < 100 and 0 < period < 100 and 0 < phase < 2 * np.pi:
        return 0.0
    return -np.inf

# Define the log posterior function
def log_posterior(params, x, y, yerr):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, x, y, yerr)

# Generate radial velocity data
import csv
with open('test.csv') as f:
    cf = csv.reader(f)
    time = []
    rv = []
    for row in cf:
        time.append(float(row[0]))
        rv.append(float(row[1]))

true_params = [0.5, 10, 5, 10, np.pi/2]  # slope, intercept, amplitude, period, phase
print(len(time), len(rv))
x = np.arange(0, 64) #np.linspace(0, 20, 100)
yerr = 0.5
y = rv #model(true_params, x) + np.random.normal(scale=yerr, size=len(x))
print(len(x), len(y))
# Set up the initial guess for the parameters
initial_params = np.array([1, 5, 2, 8, np.pi])

# Set up the emcee sampler
ndim = len(initial_params)
nwalkers = 100
nsteps = 1000

# Initialize walkers in a small Gaussian ball around the initial parameter values
p0 = initial_params + 1e-4 * np.random.randn(nwalkers, ndim)

# Set up the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(x, y, yerr))

# Run the MCMC sampler
sampler.run_mcmc(p0, nsteps, progress=True)

# Get the chains (discard burn-in)
burnin = 100
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))

# Plot the results
plt.figure(figsize=(10, 6))
plt.errorbar(x, y, yerr=yerr, fmt='o', label='Data', color='black', zorder=5, markersize=5)

# Plot some random samples from the chains
for params in samples[np.random.randint(len(samples), size=100)]:
    plt.plot(x, model(params, x), color='gray', alpha=0.1)

plt.xlabel('Time')
plt.ylabel('Radial Velocity')
plt.title('Radial Velocity Data with MCMC Fits')
plt.legend()
plt.show()


64 64
64 64


 75%|█████████████████████████████▎         | 752/1000 [00:02<00:00, 263.15it/s]